# Comparing a base model to an instruction finetuned model

![](img/base_vs_instruct.jpg)

* https://medium.com/@yananchen1116/clearance-of-the-confusion-llms-base-and-instruct-version-48d4ef402591

### Setup

In [1]:
from langchain_ollama import OllamaLLM

#### Models

* Here we will compare two versions of Meta's `llama3.2` model using the quantized version available through Ollama (See ollama.ai for details on using LLMs on your own machine)

* https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/

* https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/MODEL_CARD.md

* Huggingface card's for the two models:
  1. `llama3.2.1B` -  base model (NO FINETUNING or ALIGNMENT)
     - https://huggingface.co/meta-llama/Llama-3.2-1B
  2. `llama3.2.1B-it` - instruction finetuned
     - https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct


> **Model Architecture**: Llama 3.2 is an auto-regressive language model that uses an optimized transformer architecture. The tuned versions use supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) to align with human preferences for helpfulness and safety.

#### Setup `langchain` LLM objects for each model

* `langchain` has an `OllamaLLM` submodule to make using ollama served models easy to use and interoperable with other parts of a chain and other LLMs

In [2]:
llama3_2_instruct = OllamaLLM(
        base_url="http://10.30.16.100:11434",
        model="llama3.2:1b-instruct-q8_0")

llama3_2_base = OllamaLLM(
        base_url="http://10.30.16.100:11434",
        model="llama3.2:1b-text-q8_0")

### Compare response from base and instruct models

#### Prompts

* Use Wh-question prompts

In [3]:
prompt1 = "Who is the president of the United States?"
prompt2 = "What is the capital of France?"
prompt3 = "Why is the sky blue?"

#### `invoke` function

* All `langchain` LLM objects have an `invoke` function that will provide synchronous response from the LLM to a passed prompt. Output only given once response is complete.

In [4]:
base_resp1 = llama3_2_base.invoke(prompt1)

print(base_resp1)

 Is there a presidential election this year?
Is the US in an economic recession or on its way to one?
What's happening with the debt ceiling and is it a cause for concern right now?
Why are you so afraid that someone will be put in office who will take away all your liberties?
How much have we been lied to over the years about Iraq?


* **NOTE** this isn't really answering the question but continuing to generate text that would most likely continue after the question `'Who is the president of the United States?'` in examples seen in the pre-training data.

* Compare with the response from the **INSTRUCTION FINE-TUNED** version of the same model

In [5]:
instruct_resp1 = llama3_2_instruct.invoke(prompt1)

print(instruct_resp1)

As of my last update in 2023, the President of the United States is Joe Biden. However, please note that political positions and information can change over time due to elections, resignations, or other events that may affect the administration. If you're looking for the most current information, I recommend checking a reliable news source or the official website of the White House.


* **NOTE**
  1. Shorter response that directly relates to the question in the prompt. Answers to short questions like `'Who is the president of the United States?'` are generally only 1 or 2 sentences long.
  2. The alignment of the model provides context and time-limitation to the response of April 2023 when training was completed.
  3. Disclaimer included in response.

### Streaming response from LLM object

* `langchain` LLM objects also include a `stream` function that allow you to receive and process responses a chunk at a time.
* This can be by synchronous (wait for chunk) or asynchronous (set up a call back and receive a token at a time)

#### Responses to `What is the capital of France?` from both models

In [7]:
print(prompt2)

What is the capital of France?


* **Base `llama3.2-1b`**
  - try rerunning the cell a few times

In [11]:
for chunk in llama3_2_base.stream(prompt2):
    print(chunk, end='')

 A. Austin B. Nashville C. New York D. Washington, D.C.
A. Paris
B. Philadelphia
C. Denver
 Boston
Answer: A

* **Instruction finetuned `llama3.2-1b-Instruct`**
  - try rerunning the cell a few times

In [10]:
for chunk in llama3_2_instruct.stream(prompt2):
    print(chunk, end='')

The capital of France is Paris.

#### Responses to `Why is the sky blue?` from both models

In [12]:
print(prompt3)

Why is the sky blue?


* **Base `llama3.2-1b`**
  - try rerunning the cell a few times

In [13]:
for chunk in llama3_2_base.stream(prompt3):
    print(chunk, end='')

 What about a rainbow?
 still ask. It's because they don't fully understand or know how to answer them.
To find out, take note of how the color changes as you move towards the horizon (or the sun). As the light gets dimmer, your vision gets dimmer too. At the end is a twilight zone where all colors are mixed together and it turns into grayish blue. When all the light disappears, that's when the sky starts to turn blue.
 is blue has been debated for centuries. Scientists have tried their best to find out what causes this phenomenon but so far no one has come up with a conclusive answer.
 facts we know about it:nd some
 of three layers - the outer layer (clouds), middle layer (space between clouds), inner layer (atmosphere). The sunlight reflects off each layer depending on their color. Blue light is the most absorbed by water vapor in the atmosphere so that makes up half of its color.
2) Water vapor also absorbs some wavelengths of red, green and yellow light but not blue since there ar

* **Instruction finetuned `llama3.2-1b-Instruct`**
  - try rerunning the cell a few times

In [14]:
for chunk in llama3_2_instruct.stream(prompt3):
    print(chunk, end='')

The sky appears blue to the human eye because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that when light travels through a gas like air, it scatters in all directions, but more efficiently in the direction of the sun.

Here's what happens:

 and encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2).
 These molecules scatter the shorter, blue wavelengths of light more than the longer, red wavelengths.
 This scattering effect is more pronounced because blue light has a smaller wavelength than red light.
 As a result, about 7% of the sunlight that enters the Earth's atmosphere is scattered in all directions, leaving about 93% of it to be reflected back to our eyes from the top and bottom of the atmosphere.

 during the day, especially during sunrise and sunset when the angle of the sun's rays hits the atmosphere at a more shallow angle. The scattered light is what gives the sky its blue appearance.

Keep in mi

## Domain finetuning

- https://huggingface.co/blog/ImranzamanML/fine-tuning-1b-llama-32-a-comprehensive-article
- https://www.llama.com/docs/how-to-guides/fine-tuning/